#  Notebook overview ..

In [ ]:

#  This Notebook is part of a set that demonstrate GNN using a movie dataset.
#  About this Notebook,
#
#  .  There was a Kaggle GNN challenge circa 2019 detailed here,
#        https://www.kaggle.com/c/movie-genre-classification/data
#
#     That data is locked down, but a similarly themed dataset also on 
#     Kaggle is here,
#        https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
#
#     And this is the data set in use here.
#
#  .  1 GB, plus or minus. Most of that volume comes from reviews. The
#     movies run 30-40 MB, and the cast and crew about 190 MB.
#
#     The data is CSV, with embedded arrays of JSON.
#     To remove dependencies on GS/S3 hosted data, this program expects this
#     data to be local to the container hosting this Jupyter Notebook.
#
#     Since we host on GitHub, and GitHub has a 25 MB file size limit,
#     the total data set is now split across multiple files.
#
#  .  The existing schema for just Movies is listed here,
#  
#        10_movies_metadata.csv
#        -----------------------------------------
#           adult                      ..   False
#           belongs_to_collection      ..
#           budget                     ..   2700000
#           genres                     ..   "[{'id': 35, 'name': 'Comedy'}]"
#           homepage                   ..   http://www.animalhouse.com/
#           id                         ..   8469
#           imdb_id                    ..   tt0077975
#           original_language          ..   en
#           original_title             ..   Animal House
#           overview                   ..   "At a 1962 College, Dean Vernon Wormer is determined to expel
#                                            the entire Delta Tau Chi Fraternity, but those troublemakers
#                                            have other plans for him."
#           popularity                 ..   7.525382
#           poster_path                ..   /AuJkgAh7zAGsm7Oo3CGyDtYvzg0.jpg
#           production_companies       ..   "[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Oregon Film Factory',
#                                               'id': 13298}, {'name': 'Stage III Productions', 'id': 13300}]"
#           production_countries       ..   "[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
#           release_date               ..   1978-07-27
#           revenue                    ..   141000000
#           runtime                    ..   109.0
#           spoken_languages           ..   "[{'iso_639_1': 'en', 'name': 'English'}]"
#           status                     ..   Released
#           tagline                    ..   It was the Deltas against the rules... the rules lost!
#           title                      ..   Animal House
#           video                      ..   False
#           vote_average               ..   7.0
#           vote_count                 ..   420
#
#     From the above, we load the following into a DataFrame of Movies nodes,
#
#           id                         ..   8469
#           title                      ..   Animal House
#           genres                     ..   "[{'id': 35, 'name': 'Comedy'}]"
#           overview                   ..   "At a 1962 College, Dean Vernon Wormer is determined to expel
#                                            the entire Delta Tau Chi Fraternity, but those troublemakers
#                                            have other plans for him."
#           tagline                    ..   It was the Deltas against the rules... the rules lost!
#
#           popularity                 ..   7.525382
#           production_companies       ..   "[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Oregon Film Factory',
#                                               'id': 13298}, {'name': 'Stage III Productions', 'id': 13300}]"
#           release_date               ..   1978-07-27
#           revenue                    ..   141000000
#           runtime                    ..   109.0
#           spoken_languages           ..   "[{'iso_639_1': 'en', 'name': 'English'}]"
#           vote_average               ..   7.0
#           vote_count                 ..   420
#
#      Notice the following from above,
#
#         ..  genres is an array of JSON, with each genre being unique identified via a numeric.
#             We will take the first genre and put it into a property on each node titled, primary_genre.
#         ..  We will leave all remaining JSON untouched, stored as strings.
#
#  .  The existing schema is for Keywords is listed here,
#
#        11_keywords.csv
#        -----------------------------------------
#           id                         ..   8469
#           keywords                   ..   "[{'id': 572, 'name': 'sex'}, {'id': 2483, 'name': 'nudity'},
#                                             {'id': 3616, 'name': 'college'}, {'id': 157632, 'name': 'fraternity'},
#                                             {'id': 158507, 'name': 'gross out comedy'}, {'id': 160450, 'name': 'dean'},
#                                             {'id': 171400, 'name': 'fraternity house'}, {'id': 208983, 'name': 'probation'},
#                                             {'id': 208992, 'name': '1960s'}, {'id': 209506, 'name': 'college freshman'},
#                                             {'id': 236316, 'name': 'anarchic comedy'}]"
#
#      From the above, the following is offered,
#   
#         ..  id  joins with movie.id
#         ..  keywords.id  already enumerates keywords associated with the movies for us.
#             Super handy.
#
#  Below we continue by loading the raw data, and performing some validations on
#  statements made, assumptions, and similar.



#  Load just Movies into a DataFrame, perform analysis ..

In [ ]:

import dask.dataframe as dd
import numpy as np
   #
import json

from tabulate import tabulate

print("--")


In [ ]:

#  Load DataFrame with raw input data

l_InputFiles  = [
   "./02_Files/40_Movies_01.txt",
   "./02_Files/41_Movies_02.txt",
]

df_data = dd.read_csv(
   l_InputFiles,
   delimiter  = ",",
   skiprows   = 1,                            #  Skip the first line of each file, since it's the column headers
   dtype      = {
      "adult"                     : np.dtype(str),
      "belongs_to_collection"     : np.dtype(str),
      "budget"                    : np.dtype(str),
      "genres"                    : np.dtype(str),
      "homepage"                  : np.dtype(str),
      "id"                        : np.dtype(str),
      "imdb_id"                   : np.dtype(str),
      "original_language"         : np.dtype(str),
      "original_title"            : np.dtype(str),
      "overview"                  : np.dtype(str),
      "popularity"                : np.dtype(str),
      "poster_path"               : np.dtype(str),
      "production_companies"      : np.dtype(str),
      "production_countries"      : np.dtype(str),
      "release_date"              : np.dtype(str),
      "revenue"                   : np.dtype(str),
      "runtime"                   : np.dtype(str),
      "spoken_languages"          : np.dtype(str),
      "status"                    : np.dtype(str),
      "tagline"                   : np.dtype(str),
      "title"                     : np.dtype(str),
      "video"                     : np.dtype(str),
      "vote_average"              : np.dtype(str),
      "vote_count"                : np.dtype(str),
      },
   names      = [
      "adult", "belongs_to_collection", "budget", "genres", "homepage", "id", "imdb_id",
      "original_language", "original_title", "overview", "popularity", "poster_path",
      "production_companies", "production_countries", "release_date", "revenue", "runtime",
      "spoken_languages", "status", "tagline", "title", "video", "vote_average", "vote_count",
      ]
   )   

df_data.compute()

print("--")


In [ ]:

#  Initial look at the data, sanity check-
#

#  print(len(df_data.index))
#     #
#  print(tabulate(df_data.head(2), headers='keys', tablefmt='psql'))
#  
#  print("--")

#  Sample output,
#
#  45466
#
#  +----+---------+----------------------------------------------------------------------------------------------
#  |    | adult   | belongs_to_collection                                                                       
#  |----+---------+--------------------------------------------------------------------------------------------
#  |  0 | False   | {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
#     'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'} | 30000000 | [{'id': 16, 'name': 'Animation'},
#     {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]  | http://toystory.disney.com/toy-story |  862 |
#     tt0114709 | en                  | Toy Story        | Led by Woody, Andy's toys live happily in his room until
#        Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots
#        against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put
#        aside their differences.                                                                                             
#     |      21.9469 | /rhIRbceoE9lR4veEXuwCC2wARtG.jpg | [{'name': 'Pixar Animation Studios', 'id': 3}]                                                                                      | [{'iso_3166_1': 'US', 'name': 'United States of America'}] | 1995-10-30     | 373554033 |        81 | [{'iso_639_1': 'en', 'name': 'English'}]                                          | Released | nan                                       | Toy Story | False   |            7.7 |         5415 |


In [ ]:

#  Drop unwanted columns
#
df_data2 = df_data[["id", "title", "overview", "tagline", "budget", "genres",
   "popularity", "production_companies", "release_date", "revenue", "runtime",
   "vote_average", "vote_count", ]]

#  print(tabulate(df_data2.head(2), headers='keys', tablefmt='psql'))

print("--")


#  Check column type of genres
#
#  l_cntr = 0
#     #
#  for l_each in df_data2.iterrows():
#     l_cntr += 1
#     if (l_cntr < 3):
#        print(type(l_each[1][5]))          #  genres
#        print(     l_each[1][5] )

#  Sample output,
#
#     <class 'str'>
#     [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
#     <class 'str'>
#     [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
        

In [ ]:

#  Save the original 'genres' string as 'genres_str'.
#  Then, convert genres to a array of distionary values.
#

def f_convert_genres(i_arg1):
   try:
      l_str1   = str(i_arg1)                                                  #  Needed this, was getting odd  json.loads()  errors otherwise
      l_str2   = l_str1.replace("'", "\"")
      l_return = json.loads(l_str2)
   except:
      l_return = json.loads('[{"id": 99999999, "name": "Unknown"}]')
   return l_return


#    The reason for the if is to prevent error upon multiple
#    executions of this code.
#
if ("genres_str" not in df_data2):
   df_data2["genres_str"   ] = df_data2.genres.map(lambda x: x            )
   df_data2["genres"       ] = df_data2.genres.map(lambda x: f_convert_genres(x) )
   print("--")
    
print("--")


In [ ]:

#  Check column type of genres
#
#  l_cntr = 0
#     #
#  for l_each in df_data2.iterrows():
#     l_cntr += 1
#        #
#     if (l_cntr < 3):
#        print(type(l_each[1][5]))                     #  genres
#        print(     l_each[1][5] )
#           #
#        print(type(l_each[1][5][0]))                  #  genres, first element
#        print(     l_each[1][5][0] )
#           #
#        print(type(l_each[1][5][0]["name"]))          #  genres, first element, given key value
#        print(     l_each[1][5][0]["name"] )
        
#  Sample output,
#  
#     <class 'list'>
#     [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
#     <class 'dict'>
#     {'id': 16, 'name': 'Animation'}
#     <class 'str'>
#     Animation
#     <class 'list'>
#     [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
#     <class 'dict'>
#     {'id': 12, 'name': 'Adventure'}
#     <class 'str'>
#     Adventure


In [62]:

#  Analysis on just genres-
#
#     .  Copy just genres into a new DataFrame. 
#     .  Pivot this list of genres into separate rows.


df_genres = df_data2[["genres"]]
   #
df_genres2 = df_genres.explode("genres")

print("--")

#  l_cntr = 0
#     #
#  for l_each in df_genres2.iterrows():
#     l_cntr += 1
#        #
#     if (l_cntr < 3):
#        print(type(l_each))
#        print(     l_each )
#           #
#        print(type(l_each[1][0]))
#        print(     l_each[1][0] )
#           #
#        print(type(l_each[1][0]["name"]))
#        print(     l_each[1][0]["name"] )
#        print("")

#  Sample data,
#
#     <class 'tuple'>
#     (0, genres    {'id': 16, 'name': 'Animation'}
#     Name: 0, dtype: object)
#     <class 'dict'>
#     {'id': 16, 'name': 'Animation'}
#     <class 'str'>
#     Animation
#     
#     <class 'tuple'>
#     (0, genres    {'id': 35, 'name': 'Comedy'}
#     Name: 0, dtype: object)
#     <class 'dict'>
#     {'id': 35, 'name': 'Comedy'}
#     <class 'str'>
#     Comedy


--


In [63]:


def aaa(i_arg1):
    
   l_str1 = str(i_arg1)                                                  #  Needed this, was getting odd  json.loads()  errors otherwise
   l_str2 = l_str1.replace("'", "\"")
   l_str3 = json.loads(l_str2)


   # l_str2 = i_arg1[0]
   # l_str2 = l_str1.replace("'", "\"")
   # l_str3 = json.loads(str2)
   l_str4 = l_str2
      #
   return l_str4

#  try:
#     l_str1 = str(i_arg1)
#     l_str2 = l_str1[0][1]
#     l_return = l_str2
#  except:
#     l_return = "Unknown2"
#  return l_return

    
# df_genres2["genres"] = df_genres2.genres.apply(lambda x: aaa(x), meta("genres", "tuple") )
df_genres2["genres"] = df_genres2.genres.apply(lambda x: aaa(x), meta=("genres", "object") )

 
l_cntr = 0
   #
for l_each in df_genres2.iterrows():
   l_cntr += 1
      #
   if (l_cntr < 3):
      print(type(l_each))
      print(     l_each )
      print("")
         #
      # print(type(l_each[1][0]["name"]))
      # print(     l_each[1][0]["name"] )

        
#  After:  .apply(func, meta=('genres', 'object'))
        
        
        

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#  Building the DataFrames that will go into the graph ..

In [ ]:

#  At this point we have one DataFrame with 34,000 rows. A flat model, it has
#  the following features,
#
#     release_year
#     title
#     country
#     director
#     cast
#     genre
#     wiki_url
#     plot
#
#  We will create two graphs from the DataFrame above; 
#
#     .  A standard graph, as you might expect,
#           (Genre) - [IN] - (Movies) - [ACTED_IN, DIRECTED] - (Person)
#              and
#           (Movies) - [FROM] - (Countries)
#
#     .  And a second graph shaped specifically to service our GNN routine
#        for "node property prediction"
#
#        ..  We'll make "Genre" a property of the node, Movie
#        ..  We'll withhold 10 or so Movies for testing
#        ..  We'll simplify the Movie relationship to Person
#
#  We will wind up with several DataFrames; one for each node and edge above.
#  We will organize the activities below into (graph-1) and )graph-2).
        

#  Our standard graph, Nodes/Edges ..

In [ ]:

#  Our standard graph, Nodes

df_movies    = df_data[["release_year", "title", "wiki_url", "plot"]].compute()
print("Movies...... " + str(len(df_movies.index)))
   #
df_genres    = df_data[["genre"  ]].drop_duplicates().compute()
print("Genres...... " + str(len(df_genres.index)))
df_countries = df_data[["country"]].drop_duplicates().compute()
print("Countries... " + str(len(df_countries.index)))
   #
df_persons   = df_data[["director"]].drop_duplicates().compute()
print("Persons..... " + str(len(df_persons.index)))




#  Our standard graph, Edges


print("--")

#  Sample output
#


In [ ]:

#  Genres looks odd at 2000+ unique entries. Look deeper at that-
#
#  We'll write df_genres to a file local to the Jupyter Docker container, so we can view it in an editor

df_genres.to_csv("02_Files/40_genres.txt", index=None, sep="|")

print("--")

#  Sample output
#
#  genre
#  unknown
#  western
#  comedy
#  short
#  short action/crime western
#  short film
#  biographical
#  drama
#  adventure


In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")

In [ ]:
# ##################################################################
#
#  This section; basic graph and database setup, reset for test


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

# my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
# ##################################################################
#
#  Load from source CSV, in this case we are using the Neo4J Movie graph


In [ ]:
import dask.dataframe as dd
import numpy as np

print("--")


In [ ]:
#  Load a DataFrame from CSV, Nodes/Vertices

l_InputFile  = "./10_NMovieDB/24_nodes.txt"

df_all_nodes1 = dd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "id"        : int,
      "id"        : np.dtype('O'),
      "_labels"   : np.dtype('O'),
      # "born"      : float, 
      "born"      : np.dtype('O'),
      "name"      : np.dtype('O'),
      # "released"  : float,
      "released"  : np.dtype('O'),
      "tagline"   : np.dtype('O'),
      "title"     : np.dtype('O')
      })

print("--")


In [ ]:
#  Number of rows, two different ways

display(len(df_all_nodes1))
display(print("{} Rows".format(df_all_nodes1.shape[0].compute())))

#  Other output

display(df_all_nodes1.head(10))
display(df_all_nodes1[["born", "name"]].head(10))

In [ ]:
#  Print all rows, subsetted columns

for l_each in df_all_nodes1.iterrows():
   print(l_each[0], "   ", l_each[1]["_labels"], "   ", l_each[1]["name"])  


In [ ]:
# ##################################################################
#
#   https://www.askpython.com/python/examples/subset-a-dataframe
#   https://www.codegrepper.com/code-examples/python/convert+float+to+int+python+pandas
#   https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.assign.html


In [ ]:
#  We have some issues we want to change with our data
#
#  .  Some of the property names have a leading underscore. Change those.
#  .  Some values which should be integer, are float.
#  .  The label values are currently ";Person" and ";Movie". Let's remove those semicolons.


In [ ]:
#  Column renames

df_all_nodes2 = df_all_nodes1.rename(columns={"_id": "id", "_labels": "label"})

print("--")

In [ ]:
#  Change float values to integer, remove leading semicolon from label

df_all_nodes3 = df_all_nodes2.assign(
   # born     = lambda x: x.born.fillna(0.0).astype(int), 
   born     = lambda x: x.born.fillna(0.0).astype(str), 
   # id       = lambda x: x.id.fillna(0.0).astype(int),
   id       = lambda x: x.id.fillna(0.0).astype(str),
   # released = lambda x: x.released.fillna(0.0).astype(int),
   released = lambda x: x.released.fillna(0.0).astype(str),
   label    = lambda x: x.label.astype(str).str[1:]
   )

print("--")

In [ ]:
display(len(df_all_nodes3))
display(df_all_nodes3.head(10))

In [ ]:
# ##################################################################

In [ ]:
#  Now, populate Persons and Movies

df_persons = df_all_nodes3[df_all_nodes3["label"] == "Person"][["id", "label", "born", "name"]]

df_movies  = df_all_nodes3[df_all_nodes3["label"] == "Movie"][["id", "label", "released", "tagline", "title"]]

print("--")

In [ ]:

display(len(df_persons))
display(df_persons.head(10))
display(len(df_movies))
display(df_movies.head(10))


In [ ]:
# ##################################################################
#
#  Repeat the style of work from above, but now for edges
#
#  .  Some of the property names have a leading underscore. Change those.
#  .  Some values which should be integer, are float.


In [ ]:
#  Load a DataFrame from CSV, Edges

l_InputFile  = "./10_NMovieDB/25_edges.txt"

df_all_edges1 = dd.read_csv(
   l_InputFile,
   delimiter = ",",
   dtype = {
      # "_start"    : float,
      "_start"    : np.dtype('O'),
      # "_end"      : float, 
      "_end"      : np.dtype('O'),
      "_type"     : np.dtype('O'),
      # "rating"    : float,
      "rating"    : np.dtype('O'),
      "roles"     : np.dtype('O'),
      "summary"   : np.dtype('O')
      })

print("--")

In [ ]:
display(len(df_all_edges1))
display(df_all_edges1.head(10))


In [ ]:
df_all_edges2 = df_all_edges1.rename(columns={"_start": "START_ID", "_end": "END_ID", "_type": "TYPE",
   "rating": "RATING", "roles": "ROLES", "summary": "SUMMARY"})

df_all_edges3 = df_all_edges2.assign(
   # START_ID = lambda x: x.START_ID.fillna(0.0).astype(int), 
   START_ID = lambda x: x.START_ID.fillna(0.0).astype(str), 
   # END_ID   = lambda x: x.END_ID.fillna(0.0).astype(int),
   END_ID   = lambda x: x.END_ID.fillna(0.0).astype(str),
   # RATING   = lambda x: x.RATING.fillna(0.0).astype(int)
   RATING   = lambda x: x.RATING.fillna(0.0).astype(str)
   )

print("--")

In [ ]:
display(len(df_all_edges3))
display(df_all_edges3.head(10))

In [ ]:
#  Split out the various edges

df_reviewed = df_all_edges3[df_all_edges3["TYPE"] == "REVIEWED"][["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]]

df_wrote    = df_all_edges3[df_all_edges3["TYPE"] == "WROTE"   ][["START_ID", "END_ID", "TYPE"]]

df_produced = df_all_edges3[df_all_edges3["TYPE"] == "PRODUCED"][["START_ID", "END_ID", "TYPE"]]

df_directed = df_all_edges3[df_all_edges3["TYPE"] == "DIRECTED"][["START_ID", "END_ID", "TYPE"]]

df_follows  = df_all_edges3[df_all_edges3["TYPE"] == "FOLLOWS" ][["START_ID", "END_ID", "TYPE"]]

#  "roles" is a string similar to  '[ "a", "b", "c" ]'
#
#  This was automatically coming in as a list-
#  Cool

df_actedin  = df_all_edges3[df_all_edges3["TYPE"] == "ACTED_IN"][["START_ID", "END_ID", "TYPE", "ROLES"]]

print("--")

In [ ]:

display(len(df_reviewed))
display(df_reviewed.head(2))

display(len(df_wrote))
display(df_wrote.head(2))

display(len(df_produced))
display(df_produced.head(2))

display(len(df_directed))
display(df_directed.head(2))

display(len(df_follows))
display(df_follows.head(2))

display(len(df_actedin))
display(df_actedin.head(2))


In [ ]:
# ##################################################################

In [ ]:
#  Didn't need this; also don't know if it had any effect

# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=2)

# print("--")

In [ ]:
# ##################################################################
#
#  Finally now, load the vertices/nodes into a graph
#
#  Some hinkiness we need to work around ..
#
#     .  The Dask DataFrames here were loaded from CSV, and those CSV
#        files were found, in scope.
#        The KG DataFrame importer will reference that same file
#        pathname, and the file will not be in scope. Basically,
#        it was expected that these files be on S3/GS all along.
#        I hate to have that dependency because, just one more thing
#        to have to manage.
#
#     .  So, we'll copy the DataFrames to Dask arrays, then back into
#        a Dask DataFrame.
#        Why not just copy the DaskDataFrame ?  Currently there is only 
#        shallow copies of DataFrames.
#
#  See,
#     https://stackoverflow.com/questions/52119342/how-do-i-convert-a-dask-dataframe-into-a-dask-array
#     https://docs.dask.org/en/latest/generated/dask.dataframe.from_dask_array.html


da_persons    = df_persons.to_dask_array()
da_movies     = df_movies.to_dask_array()
   #
da_directed   = df_directed.to_dask_array()
da_reviewed   = df_reviewed.to_dask_array()
da_wrote      = df_wrote.to_dask_array()
da_produced   = df_produced.to_dask_array()
da_follows    = df_follows.to_dask_array()
da_actedin    = df_actedin.to_dask_array()


df_persons2   = dd.io.from_dask_array(da_persons,  columns=["id", "label", "born", "name"]).compute()
df_movies2    = dd.io.from_dask_array(da_movies,   columns=["id", "label", "released", "tagline", "title"]).compute()
   #
df_directed2  = dd.io.from_dask_array(da_directed, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_reviewed2  = dd.io.from_dask_array(da_reviewed, columns=["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]).compute()
df_wrote2     = dd.io.from_dask_array(da_wrote, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_produced2  = dd.io.from_dask_array(da_produced, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_follows2   = dd.io.from_dask_array(da_follows, columns=["START_ID", "END_ID", "TYPE"]).compute()
df_actedin2   = dd.io.from_dask_array(da_actedin, columns=["START_ID", "END_ID", "TYPE", "ROLES"]).compute()

print("--")



In [ ]:
from katana_enterprise.remote import import_data

print("--")


In [ ]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   # Person
   #
   df_importer.nodes_dataframe(
      df_persons2[["id", "label", "born", "name"]],
      id_column  = "id",
      id_space   = "Person"
      )
   #  Movie
   #
   df_importer.nodes_dataframe(
      df_movies2[["id", "label", "title", "tagline"]],
      id_column  = "id",
      id_space   = "Movie"
      )  
    
   #  DIRECTED
   #
   df_importer.edges_dataframe(
      df_directed2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "DIRECTED"
      )
   #  REVIEWED
   #
   df_importer.edges_dataframe(
      df_reviewed2[["START_ID", "END_ID", "TYPE", "RATING", "SUMMARY"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "REVIEWED"
      )
   #  WROTE
   #
   df_importer.edges_dataframe(
      df_wrote2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "WROTE"
      )
   #  PRODUCED
   #
   df_importer.edges_dataframe(
      df_produced2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "PRODUCED"
      )
   #  FOLLOWS
   #
   df_importer.edges_dataframe(
      df_follows2[["START_ID", "END_ID", "TYPE"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "FOLLOWS"
      )
   #  ACTEDIN
   #
   df_importer.edges_dataframe(
      df_actedin2[["START_ID", "END_ID", "TYPE", "ROLES"]],
      source_id_space      = "Person",
      destination_id_space = "Movie",
      source_column        = "START_ID",
      destination_column   = "END_ID",
      type                 = "ACTEDIN"
      )

   df_importer.execute()
    
    

In [ ]:

l_result1 = my_graph.query("""

   MATCH (a) 
   WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """)

display(print(l_result1))


In [ ]:

l_result1 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """)

display(print(l_result1))


In [ ]:

result = my_graph.query("""

   MATCH (x) -[r]-> (a)
   RETURN x, r AS rel, a
   
   """,
   contextualize=True)

result.view()


# Output a graph as a a local file

In [ ]:

#  Formatting could use a little work, but the concept is here ..

l_result1 = my_graph.query("""
   MATCH (n: Person) 
   RETURN n
   """)
      #
# display(print(l_result1))

l_result2 = my_graph.query("""
   MATCH (n) - [r: WROTE] -> (m)
   RETURN r
   """)
      #
# display(print(l_result2))


In [ ]:

l_nodes = []
   #
for l_node in l_result1.iterrows():
   l_nodes.append(l_node)

l_file = open("nodes.txt", "w")
l_file.write(str(l_nodes))
l_file.close()


l_edges = []
   #
for l_edge in l_result2.iterrows():
   l_edges.append(l_edge)

l_file = open("edges.txt", "w")
l_file.write(str(l_edges))
l_file.close()


print("--")
